In [ ]:
!conda install geopandas

In [ ]:
pip install geopy

In [ ]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import numpy as np 
import pandas as pd

In [ ]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
# we use Beautiful soup for parsing the wiki page
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# look for the table we need, with class wikitable sortable
table_w = soup.find('table','wikitable sortable')

In [ ]:
# look for the cell headers with <th> element
th_1 = table_w.tr.find_all("th")
titulos = list()
for element in th_1:
    element = str(element)
    element = element.replace('<th>','').replace('</th>','').replace('\n','')
    titulos.append(element)
titulos

In [ ]:
# look for the table data with <tr> 
th_2 = table_w.find_all("tr")
# we delete the first list that contains the headers
th_2 = th_2[1:]
# convert the html code to text str, we edit the text we dont need "<tr>, </tr> and \n". We split each row with <td> 
# select the only elements in the list we are interested
data = list()
for lista in th_2:
    lista = str(lista)
    lista = lista.replace('<tr>','').replace('</tr>','').replace('</td>','').replace('\n','')
    lista = lista.split('<td>')
    lista = lista[1:4]
    data.append(lista)

In [ ]:
# we convert list to a dataframe
data_1 = pd.DataFrame(data)
# add the titles and delete a 4th column generated by error
data_1.columns = titulos
# we drop Borough which are not assigned
data_1.set_index('Borough', inplace=True) 
data_1.drop(["Not assigned"], inplace = True) 
data_1.reset_index(inplace=True)

In [ ]:
# control if we have a 'Not assigned' Neighborhood
data_1_no = data_1[data_1['Neighborhood']=='Not assigned']
print(data_1_no.shape)

In [ ]:
# check 
data_1.shape

In [ ]:
# check the look
data_1

In [ ]:
# create a list of postal codes
postal_code = data_1['Postal Code'].to_list()

In [ ]:
locator = Nominatim(user_agent=”myGeocoder”)
location = locator.geocode(“Champ de Mars, Paris, France”)
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

In [ ]:
lat_lng = list()
for codes in postal_code:
    g = geocoder.google('Canada, Toronto, '+codes)
    lat_lng_coords = g.latlng
    print(lat_lng_coords)